In [3]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Dense, SimpleRNN # for creating regular densely-connected NN layers and RNN layers

# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version
import math # to help with data reshaping of the data

# Sklearn
import sklearn # for model evaluation
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.model_selection import train_test_split # for splitting the data into train and test samples
from sklearn.metrics import mean_squared_error # for model evaluation metrics
from sklearn.preprocessing import MinMaxScaler # for feature scaling

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

Tensorflow/Keras: 2.12.0
pandas: 1.4.2
numpy: 1.22.2
sklearn: 1.2.2
plotly: 5.15.0


In [4]:
pd.options.display.max_columns=50
df=pd.read_csv('ds/dataset.csv', encoding='utf-8')
df['GDate']=pd.to_datetime(df['GDate'],format="%Y%m%d")
df

,LVal18AFC,PDate,GDate,NumberTrade,VolumeTrade,Value,ClosePrice,LastPrice,PriceMin,PriceMax,PriceFirst,PriceYesterday
0,وبملت,13991227,2021-03-17,7213,160731993,6.929920e+11,4310,4330,4230,4380,4320,4210
1,وبملت,13991226,2021-03-16,13563,499852655,2.104940e+12,4210,4320,4090,4380,4090,4170
2,وبملت,13991225,2021-03-15,11046,362136753,1.511250e+12,4170,4240,3990,4280,4020,4070
3,وبملت,13991224,2021-03-14,11100,270006098,1.098370e+12,4070,4020,3990,4150,4080,3940
4,وبملت,13991223,2021-03-13,9356,281339188,1.109270e+12,3940,3990,3730,3990,3780,3770
...,...,...,...,...,...,...,...,...,...,...,...,...
1173,وبملت,13950111,2016-03-30,2585,37220601,9.539019e+10,2563,2550,2510,2609,2510,2510
1174,وبملت,13950110,2016-03-29,1795,25419978,6.379931e+10,2510,2514,2491,2520,2504,2508
1175,وبملت,13950109,2016-03-28,2349,38105647,9.555878e+10,2508,2513,2450,2534,2499,2498
1176,وبملت,13950108,2016-03-27,2822,44216051,1.104410e+11,2498,2470,2451,2556,2501,2440


In [5]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df['GDate'], 
                         y=df['ClosePrice'],
                         mode='lines',
                         name='مقدار معامله',
                         opacity=0.8,
                         line=dict(color='black', width=1)
                        ))
fig.update_layout(dict(plot_bgcolor = 'white'))
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='تاریخ'
                )
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='مقدار معامله'
                )

fig.update_layout(title=dict(text="مقدار فروش وبملت", 
                             font=dict(color='black')))
fig.show()

In [6]:
def prep_data(datain, time_step):
    # 1. y-array  
    # First, create an array with indices for y elements based on the chosen time_step
    y_indices = np.arange(start=time_step, stop=len(datain), step=time_step)
    # Create y array based on the above indices 
    y_tmp = datain[y_indices]
    # 2. X-array  
    # We want to have the same number of rows for X as we do for y
    rows_X = len(y_tmp)
    # Since the last element in y_tmp may not be the last element of the datain, 
    # let's ensure that X array stops with the last y
    X_tmp = datain[range(time_step*rows_X)]
    # Now take this array and reshape it into the desired shape
    X_tmp = np.reshape(X_tmp, (rows_X, time_step, 1))
    return X_tmp, y_tmp

In [7]:
X=df[['ClosePrice']]
scaler=MinMaxScaler()
X_scaled=scaler.fit_transform(X)
train_data, test_data = train_test_split(X_scaled, test_size=0.2, shuffle=False)
time_step = 7
X_train, y_train = prep_data(train_data, time_step)
X_test, y_test = prep_data(test_data, time_step)

In [8]:
model = Sequential(name="First-RNN-Model") # Model
model.add(Input(shape=(time_step,1), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model.add(SimpleRNN(units=1, activation='tanh', name='Hidden-Recurrent-Layer')) # Hidden Recurrent Layer, Tanh(x) = sinh(x)/cosh(x) = ((exp(x) - exp(-x))/(exp(x) + exp(-x)))
model.add(Dense(units=1, activation='tanh', name='Hidden-Layer')) # Hidden Layer, Tanh(x) = sinh(x)/cosh(x) = ((exp(x) - exp(-x))/(exp(x) + exp(-x)))
model.add(Dense(units=1, activation='linear', name='Output-Layer')) # Output Layer, Linear(x) = x
model.compile(optimizer='adam', # default='rmsprop', an algorithm to be used in backpropagation
              loss='mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
             )


In [9]:
model.fit(X_train, y_train, batch_size=1,epochs=20,)

Epoch 1/20
134/134 [==============================] - 1s 2ms/step - loss: 0.0160 - mean_squared_error: 0.0160 - mean_absolute_error: 0.0510
Epoch 2/20
134/134 [==============================] - 0s 2ms/step - loss: 0.0124 - mean_squared_error: 0.0124 - mean_absolute_error: 0.0400
Epoch 3/20
134/134 [==============================] - 0s 2ms/step - loss: 0.0104 - mean_squared_error: 0.0104 - mean_absolute_error: 0.0403
Epoch 4/20
134/134 [==============================] - 0s 2ms/step - loss: 0.0093 - mean_squared_error: 0.0093 - mean_absolute_error: 0.0439
Epoch 5/20
134/134 [==============================] - 0s 2ms/step - loss: 0.0087 - mean_squared_error: 0.0087 - mean_absolute_error: 0.0418
Epoch 6/20
134/134 [==============================] - 0s 2ms/step - loss: 0.0081 - mean_squared_error: 0.0081 - mean_absolute_error: 0.0401
Epoch 7/20
134/134 [==============================] - 0s 2ms/step - loss: 0.0077 - mean_squared_error: 0.0077 - mean_absolute_error: 0.0404
Epoch 8/20
134/134 [

In [10]:
pred_train = model.predict(X_train)
# Predict the result on test data
pred_test = model.predict(X_test)

2/2 [==============================] - 0s 3ms/step


In [11]:
print("")
print('-------------------- Model Summary --------------------')
model.summary() # print model summary
print("")
print('-------------------- Weights and Biases --------------------')
print("Note, the last parameter in each layer is bias while the rest are weights")
print("")
for layer in model.layers:
    print(layer.name)
    for item in layer.get_weights():
        print("  ", item)
print("")
print('---------- Evaluation on Training Data ----------')
print("MSE: ", mean_squared_error(y_train, pred_train))
print("")

print('---------- Evaluation on Test Data ----------')
print("MSE: ", mean_squared_error(y_test, pred_test))
print("")


-------------------- Model Summary --------------------
Model: "First-RNN-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Hidden-Recurrent-Layer (Sim  (None, 1)                3         
 pleRNN)                                                         
                                                                 
 Hidden-Layer (Dense)        (None, 1)                 2         
                                                                 
 Output-Layer (Dense)        (None, 1)                 2         
                                                                 
Total params: 7
Trainable params: 7
Non-trainable params: 0
_________________________________________________________________

-------------------- Weights and Biases --------------------
Note, the last parameter in each layer is bias while the rest are weights

Hidden-Recurrent-Layer
   [[-0.20863785]]
   [[0.99501306]]
 

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.array(range(0,len(y_test))),
                         y=scaler.inverse_transform(y_test).flatten(),
                         mode='lines',
                         name='قیمت بسته شده - Actual (Test)',
                         opacity=0.8,
                         line=dict(color='black', width=1)
                        ))
fig.add_trace(go.Scatter(x=np.array(range(0,len(pred_test))),
                         y=scaler.inverse_transform(pred_test).flatten(),
                         mode='lines',
                         name='قیمت بسته شده - Predicted (Test)',
                         opacity=0.8,
                         line=dict(color='red', width=1)
                        ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='مشاهدات'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='قیمت بسته شده'
                )

# Set figure title
fig.update_layout(title=dict(text="وبملت RNN", 
                             font=dict(color='black')),
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                 )

fig.show()